# Chapter 3

## 3.1

In [6]:
(define (make-accumulator acc)
  (lambda (x) (set! acc (+ acc x)) acc))

In [7]:
(define A (make-accumulator 5))

In [8]:
(A 10)

15

In [10]:
(A 10)

25

## 3.2

In [15]:
(define (make-monitored f)
  (define count 0)
  (define (mf m)
    (cond ((eq? m 'how-many-calls?) count)
          ((eq? m 'reset-count) (set! count 0))
          (else 
           (set! count (+ count 1))
           (f m))))
  mf)

In [30]:
(define s (make-monitored sqrt))

In [31]:
(s 100)

10.0

In [32]:
(s 'how-many-calls?)

1

In [33]:
(s 25)

5.0

In [34]:
(s 'how-many-calls?)

2

In [35]:
(s 'reset-count)
(s 'how-many-calls?)

0

## 3.3

In [79]:
(define (make-account balance password)
  (define (withdraw amount)
    (if (>= balance amount)
        (begin (set! balance
                     (- balance amount))
               balance)
        "Insufficient funds"))
  (define (deposit amount)
    (set! balance (+ balance amount))
    balance)
  
  (define (dispatch m)
    (cond ((eq? m 'withdraw) withdraw)
          ((eq? m 'deposit) deposit)
          (else (error "Unkown request:
                       MAKE-ACCOUNT" m))))
  
  (define (authorise pass m)
    (if (eq? pass password)
        (dispatch m)
        (lambda (amount) "Incorrect password")))
    
  authorise)

In [80]:
(define acc
  (make-account 150 '1234safepassword))

((acc '1234safepassword 'withdraw) 100)

50

In [42]:
((acc 'wrongpassword 'deposit) 100)

"Incorrect password"

## 3.4

In [71]:
(define (make-account balance password)
  (define failed-attempts 0)
  (define (call-the-police) "Too many failed attempts, police on the way.")
  (define (withdraw amount)
    (if (>= balance amount)
        (begin (set! balance
                     (- balance amount))
               balance)
        "Insufficient funds"))
  (define (deposit amount)
    (set! balance (+ balance amount))
    balance)
  
  (define (dispatch m)
    (cond ((eq? m 'withdraw) withdraw)
          ((eq? m 'deposit) deposit)
          (else (error "Unkown request:
                       MAKE-ACCOUNT" m))))
  
  (define (authorise pass m)
    (if (eq? pass password)
        (begin (set! failed-attempts 0)
               (dispatch m))
        (begin (set! failed-attempts (+ 1 failed-attempts))
               (lambda (amount) "Incorrect password"))))
  
  (define (limit-attempts pass m)
    (if (>= failed-attempts 7)
         (lambda (x) (call-the-police))
        (authorise pass m)))
    
  limit-attempts)

In [72]:
(define acc
  (make-account 150 '1234safepassword))

In [73]:
((acc '1234safepassword 'withdraw) 100)

50

In [74]:
((acc 'wrongpassword 'withdraw) 100)
((acc 'wrongpassword 'withdraw) 100)
((acc 'wrongpassword 'withdraw) 100)
((acc 'wrongpassword 'withdraw) 100)
((acc 'wrongpassword 'withdraw) 100)
((acc 'wrongpassword 'withdraw) 100)
((acc 'wrongpassword 'withdraw) 100)

((acc '1234safepassword 'withdraw) 100)

"Too many failed attempts, police on the way."

## 3.5

In [15]:
(define (square x)
  (* x x))

(define (monte-carlo trials experiment)
  (define (iter trials-remaining trials-passed)
    (cond ((= trials-remaining 0)
           ( / trials-passed trials))
          ((experiment)
           (iter (- trials-remaining 1)
                 (+ trials-passed 1)))
          (else
           (iter (- trials-remaining 1)
                 trials-passed))))
  (iter trials 0))

(define (random-in-range low high)
  (let ((range (- high low)))
    (+ low (random range))))

(define (make-point x y) (cons x y))
(define (get-x point) (car point))
(define (get-y point) (cdr point))

(define (make-circle center-point radius)
   (list center-point radius))
(define (get-center circle) (car circle))
(define (get-radius circle) (cadr circle))

(define (make-rect p1 p2) (list p1 p2))
(define (width-rect rect)
  (abs (- (get-x (car rect))
          (get-x (cadr rect)))))
(define (height-rect rect)
  (abs (- (get-y (car rect))
          (get-y (cadr rect)))))
(define (area-rect rect)
  (* (width-rect rect) (height-rect rect)))

(define (rand-point-in-rect rect)
  (make-point (random-in-range (get-x (car rect)) 
                               (get-x (cadr rect)))
              (random-in-range (get-y (car rect))
                               (get-y (cadr rect)))))

(define (point-in-circle? point circle)
  (let ((circle-center-x (get-x (get-center circle)))
        (circle-center-y (get-y (get-center circle))))
    (<= (+ (square (- (get-x point) circle-center-x))
           (square (- (get-y point) circle-center-y)))
        (square (get-radius circle)))))

;; P should take a rectangle as input and test whether a random
;; point within that rectangle is within a desired circle
;; the circle is internal to P
(define (estimate-integral P bounds-rect trials)
  (monte-carlo trials (P bounds-rect)))

(define unit-rect (make-rect (make-point -1.0 -1.0)
                             (make-point 1.0 1.0)))
(define unit-circle (make-circle (make-point 0 0) 1.0))


(define (estimate-pi-integral trials)
  (define (rand-point-in-unit-circle bounds-rect) 
    (lambda ()
      (point-in-circle? (rand-point-in-rect bounds-rect)
                        unit-circle)))
  (* (area-rect unit-rect) 
     (estimate-integral rand-point-in-unit-circle unit-rect trials))) 


This takes a long time to run -> markdown cell to prevent accidentally running again.
```scheme
;;(estimate-pi-integral 100000)
;; 3.144
```

## 3.6

In [62]:
;; naive implementations for testing
;; +1 to prevent returning 0 -> random takes positive value
(define (random-init) (+ 1 (random 1000))) 
(define (rand-update x) (+ 1 (random x)))

(define rand
  (let ((x (random-init)))
    (lambda (m)
      (cond ((eq? m 'generate)
             (set! x (rand-update x)) x)
            ((eq? m 'reset)
             (lambda (new-value)
               (set! x new-value) x))
            (else (error "Unkown method:
                         RAND" m))))))

In [63]:
(rand 'generate)

338

In [64]:
((rand 'reset) 10)

10

## 3.7

In [74]:
(define (make-account balance password)
  (define (withdraw amount)
    (if (>= balance amount)
        (begin (set! balance
                     (- balance amount))
               balance)
        "Insufficient funds"))
  (define (deposit amount)
    (set! balance (+ balance amount))
    balance)
  
  (define (dispatch m)
    (cond ((eq? m 'withdraw) withdraw)
          ((eq? m 'deposit) deposit)
          (else (error "Unkown request:
                       MAKE-ACCOUNT" m))))
  
  (define (correct-password? pass)
    (eq? pass password))
  
  (define (authorise pass m)
    (if (correct-password? pass)
        (dispatch m)
        (lambda (amount) "Incorrect password")))
  authorise)

(define (make-joint acc password joint-password)
  (define (dispatch pass m)
    (cond ((not (eq? pass joint-password))
           (error "Incorrect password"))
          (else (acc password m))))
  dispatch)

In [75]:
(define peter-acc (make-account 200 'peters-password))
(define paul-acc
  (make-joint peter-acc
              'peters-password
              'pauls-password))

((peter-acc 'peters-password 'withdraw) 10)

190

In [76]:
((paul-acc 'pauls-password 'withdraw) 10)

180

In [77]:
((peter-acc 'peters-password 'withdraw) 10)

170

In [78]:
((paul-acc 'pauls-password 'deposit) 100)

270

In [81]:
((peter-acc 'peters-password 'withdraw) 10)

260

## 3.8
`f` should return it's argument if it has been previously called, else it should return `0`.

In [99]:
(define f
  (let ((called? #f))
    (lambda (x)
      (if called?
          0
          (begin
           (set! called? #t)
           x)))))

(+ (f 0) (f 1))

0

In [100]:
;; re-define to clear out internal state
(define f
  (let ((called? #f))
    (lambda (x)
      (if called?
          0
          (begin
           (set! called? #t)
           x)))))
;; call opposite way round to simulate changing evaluation order
(+ (f 1) (f 0))

1

## 3.9

### Recursive Factorial
```scheme
(define (factorial n)
  (if (= n 1)
      1
      (* n (factorial (- n 1)))))
```

#### Environment Structure
``` 
         __________________
global  |                  |
env --->|factorial: *      |
        |           |      |
        |___________|______|
                    |
            parameters: n
            body: (if (= n 1) 1 (* n (factorial (-n 1))))

(factorial 6)
             _______      ^
            |       |     | global env
    E1 ---> | n : 6 |_____|
            |_______|

(* 6 (factorial 5))
             _______      ^
            |       |     | global env
    E2 ---> | n : 5 |_____|
            |_______|
            
(* 5 (factorial 4))
             _______      ^
            |       |     | global env
    E3 ---> | n : 4 |_____|
            |_______|

(* 4 (factorial 3))
             _______      ^
            |       |     | global env
    E4 ---> | n : 3 |_____|
            |_______|

(* 3 (factorial 2))
             _______      ^
            |       |     | global env
    E5 ---> | n : 2 |_____|
            |_______|

(* 2 (factorial 1))
             _______      ^
            |       |     | global env
    E2 ---> | n : 1 |_____|
            |_______|
1
```

### Iterative Factorial
```scheme
(define (factorial n)
  (fact-iter 1 1 n))

(define (fact-iter product 
                   counter 
                   max-count)
  (if (> counter max-count)
      product
      (fact-iter (* counter product)
                 (+ counter 1)
                 max-count)))
```

#### Environment Structure
```
         ________________________________
global  |                                |
env --->|factorial: *                    |
        |fact-iter: |                 *  |
        |___________|_________________|__|
                    |                 |
            parameters: n             |
            body: (fact-iter 1 1 n)   parameters: (product counter max-count)
                                      body: (if (> counter max-count)
                                                  product
                                                  (fact-iter (* counter product)
                                                             (+ counter 1)
                                                             max-count))
(factorial 6)
             _______      ^
            |       |     | global env
    E1 ---> | n : 6 |_____|
            |_______|

(fact-iter 1 1 6)
             ______________      ^
            | product: 1   |     | global env
    E2 ---> | counter : 1  |_____|
            | max-count: 6 |
            |______________|

(fact-iter 1 2 6)
             ______________      ^
            | product: 1   |     | global env
    E3 ---> | counter : 2  |_____|
            | max-count: 6 |
            |______________|
            
(fact-iter 2 3 6)
             ______________      ^
            | product: 2   |     | global env
    E3 ---> | counter : 3  |_____|
            | max-count: 6 |
            |______________|

(fact-iter 6 4 6)
             ______________      ^
            | product: 6   |     | global env
    E3 ---> | counter : 4  |_____|
            | max-count: 6 |
            |______________|
            
(fact-iter 24 5 6)
             ______________      ^
            | product: 24  |     | global env
    E3 ---> | counter : 5  |_____|
            | max-count: 6 |
            |______________|

(fact-iter 120 6 6)
             ______________      ^
            | product: 120 |     | global env
    E3 ---> | counter : 6  |_____|
            | max-count: 6 |
            |______________|

(fact-iter 720 7 6)
             ______________      ^
            | product: 720 |     | global env
    E3 ---> | counter : 7  |_____|
            | max-count: 6 |
            |______________|
720
```

## 3.10

`(define W1 (make-withdraw 100))`:

Evaluating `make-withdraw` creates a new frame - `Frame A` within which `initial-amount` is bound to the value `100`.

Evaluating the lambda function within `make-withdraw` creates a new frame `Frame B` within which `balance` is bound to the value of `initial-amount` from `Frame A`.

```
         _______________________
global->| make-withdraw : *     |
env     | W1 :  *         |     |
         -------|---^-----|---^-
                |   |     |   |
                |   |     parameters: initial-amount
                |   |     body: ((lambda (balance) ((...))) initial-amount)
                |   |
                |  _|___Frame_A__________
                | | initial-amount : 100 | E1
                |  -^--------------------
                |   |
                |  _|___Frame_B___
                | | balance : 100 | E2
                |  -^-------------
                |   |
                parameters: balance
                body: (lambda (amount) (if (>= balance amount) ... ))
```


`(W1 50)`:

The call to `set!` within `Frame B` will affect the value of `balance` within `Frame B` and will not affect the value of `initial-amount` within `Frame A`.

```
         _______________________
global->| make-withdraw : *     |
env     | W1 :  *         |     |
         -------|---^-----|---^-
                |   |     |   |
                |   |     parameter: initial-amount
                |   |     body: ((lambda (balance) ((...))) initial-amount)
                |   |
                |  _|___Frame_A__________
                | | initial-amount : 100 | E1
                |  -^--------------------
                |   |
                |  _|__Frame_B___
                | | balance : 50 | E2
                |  -^------------
                |   |
                |  _|__Frame_C___
                | | amount : 50  | E3
                |  -^------------
                |   |
                parameters: amount
                body: (if (>= balance amount) ... )
```

## 3.12

Missing `<response>`s in order:
1. `(b)`
    - `(define x (list 'a 'b))` binds `x` to the value `(a b)`
    - `(cdr x)` simply returns the `cdr` of the list `(a b)`
2. `(b c d)`
    - `(define y (list 'c 'd))` binds `y` the the value `(c d)`
    - `(define w (append! x y))` binds the result of `(append! x y)` to `w`
    - `(append! x y)` *mutates* `x` such that the list `y` is appended onto it and *returns* a pointer to `x`.
    - `w` now points to the mutated value of `x`
    - `(set-cdr! (last-pair x) y)` within `append!` will set the `cdr` of the last pair in `x` to the list `y`
    - `(last-pair x)` evalutes to `(last-pair (a b)) ` which returns `(b)`
    - `(set-cdr! (last-pair x) y)` evaluates to `(set-cdr! (b) (c d))`, *mutating* `(b)` to produce `(b c d)`

### Demonstration

In [2]:
(define (append! x y)
  (set-cdr! (last-pair x) y)
  x)

(define (last-pair x)
  (if (null? (cdr x))
      x
  (last-pair (cdr x))))

(define x (list 'a 'b))
(define y (list 'c 'd))
(define z (append x y))

z

(a b c d)

In [124]:
(cdr x)

(b)

In [125]:
(define w (append! x y))
w

(a b c d)

In [126]:
(cdr x)

(b c d)

## 3.13

In [3]:
(define (make-cycle x)
  (set-cdr! (last-pair x) x)
  x)

(define z (make-cycle (list 'a 'b 'c)))

```
        ___________________
       |                   |
       v                   |
z --> [.|.] --> [.|.] --> [.|/]
       |         |         |
       v         v         v
      'a        'b        'c
```

Computing `(last-pair z)` will cause an *infinite loop* as the `cdr` of the last item in `z` is a pointer back to the first item.
- The structure is **not null terminated**.

## 3.14
`mystery` *reverses* a list

In [136]:
(define (mystery x)
  (define (loop x y)
    (if (null? x)
        y
        (let ((temp (cdr x)))
          (set-cdr! x y)
          (loop temp x))))
  (loop x '()))

(mystery '(1 2 3 4))

(4 3 2 1)

Used capitals for `v` and `w` for clarity with ASCII arrows:

```scheme
(define V (list 'a 'b 'c 'd))
```
```
V --> [.|.] --> [.|.] --> [.|.] --> [.|/]
       |         |         |         |         
       v         v         v         v
      'a        'b        'c        'd
```
```scheme
(define W (mystery v))
```

```                   
                                     V
                                     |
                                     v
W --> [.|.] --> [.|.] --> [.|.] --> [.|/]
       |         |         |         |         
       v         v         v         v
      'd        'c        'b        'a
```

In [150]:
(define V (list 'a 'b 'c 'd))
V

(a b c d)

In [151]:
(define W (mystery V))
W

(d c b a)

In [152]:
V

(a)

## 3.15
```scheme
(define x (list 'a 'b))
```
```
x --> [.|.] --> [.|/]
       |         |
       v         v
      'a        'b
```
```scheme
(define z1 (cons x x))
```
```
z1 --> [.|.]
        | |
        v v
 x --> [.|.] --> [.|/]
        |         |
        v         v
       'a        'b
```
```scheme
(define z2 (cons (list 'a 'b) 
                 (list 'a 'b)))
```
```
z2 --> [.|.] --> [.|.] --> [.|/]
        |         |         |
        |         v         v
        |        'a        'b
        |         ^         ^
        |         |         |
        |------> [.|.] --> [.|/]
```
```scheme
(define (set-to-wow! x)
  (set-car! (car x) 'wow)
  x)

(set-to-wow! z1)
```
```
z1 --> [.|.]
        | |
        v v
 x --> [.|.] --> [.|/]
        |         |
        v         v
       'wow      'b
```
```scheme
(set-to-wow! z2)
```
```
z2 --> [.|.] --> [.|.] --> [.|/]
        |         |         |
        |         v         v
        |        'a        'b
        |                   ^
        |                   |
        |------> [.|.] --> [.|/]
                  |
                  v
                 'wow
```

## 3.16

In [38]:
(define (count-pairs x)
  (if (not (pair? x))
      0
  (+ (count-pairs (car x))
     (count-pairs (cdr x))
     1)))

(define l3 (list 'a 'b 'c))

```
l3 --> [.|.] --> [.|.] --> [.|/]
       v|         |         |
        v         v         v
       'a        'b        'c
```

In [39]:
(count-pairs l3)

3

In [41]:
(define l4 (list 'a 'b))
(define l4b (list 'c))
(set-car! l4 l4b)
(set-car! (cdr l4) l4b)

```scheme
(define l4 (list 'a 'b))
(define l4b (list c))
```
```
l4 --> [.|.] --> [.|/]
        |         |
        v         v
       'a        'b
       
l4b --> [.|/]
         |
         v
        'c
```
```scheme
(set-car! l4 l4b)
```
```       
l4 --> [.|.] --> [.|/]
        |         |
        v         v
       l4b       'b
        |
        v
       [.|/]
        |
        v
       'c
```
```scheme
(set-car! (cdr l4) (l4b))
```
```       
l4 --> [.|.] --> [.|/]
        |         |
        v         |
       l4b <-------
        |
        v
       [.|/]
        |
        v
       'c
```

In [21]:
(count-pairs l4a)

4

In [26]:
(define l7 (list 'a))
(define l7a (list 'b))
(define l7b (list 'c))
(set-car! l7a l7b)
(set-cdr! l7a l7b)
(set-car! l7 l7a)
(set-cdr! l7 l7a)

```scheme
(define l7 (list 'a))
(define l7a (list 'b))
(define l7b (list 'c))
```
```
l7 --> [.|/]
        |
        v
       'a
       
l7a --> [.|/]
         |
         v
        'b
       
l7b --> [.|/]
         |
         v
        'c
```
```scheme
(set-car! l7a l7b)
```
```
l7a --> [.|/]
         |
         v
        l7b
         |
         v
        [.|/]
         |
         v
        'c
```
```scheme
(set-cdr! l7a l7b)
```
```
l7a --> [.|.] --
         |     |
         v     |
        l7b <---
         |
         v
        [.|/]
         |
         v
        'c
```
```scheme
(set-car! l7 l7a)
```
```
l7 --> [.|/]
        |
        v
        l7a --> [.|.] --
                 |     |
                 v     |
                l7b <---
                 |
                 v
                [.|/]
                 |
                 v
                'c
```
```scheme
(set-cdr! l7 l7a)
```
```
l7 --> [.|.]
        | |
        v v
        l7a --> [.|.]
                 | |
                 v v
                 l7b 
                  |
                  v
                 [.|/]
                  |
                  v
                 'c
```

In [27]:
(count-pairs l7)

7

## 3.17

In [74]:
(define (count-pairs x)
  (let ((counted-pairs '()))
    (define (iter x)
      (cond ((not (pair? x)) 0)
            ((memq x counted-pairs) 0)
            (else (set! counted-pairs (cons x counted-pairs))
                  (+ (iter (car x))
                     (iter (cdr x))
                     1))))
    (iter x)))

In [75]:
;; test with same lists from 3.16
(define l3 (list 'a 'b 'c))

(define l4 (list 'a 'b))
(define l4b (list 'c))
(set-car! l4 l4b)
(set-car! (cdr l4) l4b)

(define l7 (list 'a))
(define l7a (list 'b))
(define l7b (list 'c))
(set-car! l7a l7b)
(set-cdr! l7a l7b)
(set-car! l7 l7a)
(set-cdr! l7 l7a)

In [76]:
(count-pairs l3)

3

In [77]:
(count-pairs l4)

3

In [78]:
(count-pairs l7)

3

## 3.18

In [5]:
(define (contains-cycle? x)
  (let ((visited '()))
    (define (iter x)
      (cond ((not (pair? x)) #f)
            ((memq (cdr x) visited) #t)
            (else (set! visited (cons x visited))
                  (iter (cdr x)))))
    (iter x)))

In [6]:
;; make-cycle from 3.13
(contains-cycle? (make-cycle (list 'a 'b 'c)))

#t

In [10]:
(contains-cycle? (list 'a 'b 'c))

#f

In [12]:
(define l (list 'a 'b 'c))
(set-cdr! (cdr (cdr l)) l)
(contains-cycle? l)

#t

In [16]:
(define l (list 1 2 3 4 5))
(set-car! (cdddr l) (cddr l))
(contains-cycle? l)

#f

## 3.19

In [36]:
;; implementation of Floyd's cycle-finding algorithm (tortois and hare)
(define (contains-cycle? x)
  (define (cdr-or-null l)
    (if (pair? l)
        (cdr l)
        '()))
  (define (iter slow fast)
    (cond ((not (pair? slow)) #f)
          ((not (pair? fast)) #f)
          ((eq? slow fast) #t)
          ((eq? slow (cdr-or-null fast)) #t)
          (else (iter (cdr-or-null slow) 
                      (cdr-or-null (cdr-or-null fast))))))
  (iter (cdr-or-null x)
        (cdr-or-null (cdr-or-null x))))

In [37]:
(contains-cycle? (make-cycle (list 'a 'b 'c)))

#t

In [38]:
(contains-cycle? (list 'a 'b 'c))

#f

In [39]:
(define l (list 'a 'b 'c))
(set-cdr! (cdr (cdr l)) l)
(contains-cycle? l)

#t

In [40]:
(define l (list 1 2 3 4 5))
(set-car! (cdddr l) (cddr l))
(contains-cycle? l)

#f

## 3.20

In [6]:
;; renamed to avoid overriding builtin cons
(define (proc-cons x y)
  (define (set-x! v) (set! x v))
  (define (set-y! v) (set! y v))
  (define (dispatch m)
    (cond ((eq? m 'car) x)
          ((eq? m 'cdr) y)
          ((eq? m 'set-car!) set-x!)
          ((eq? m 'set-cdr!) set-y!)
          (else (error "Undefined
                       operation: CONS" m))))
  dispatch)

(define (proc-car z) (z 'car))
(define (proc-cdr z) (z 'cdr))

(define (proc-set-car! z new-value)
  ((z 'set-car!) new-value)
  z)

(define (proc-set-cdr! z new-value)
  ((z 'set-cdr!) new-value)
  z)

Description of process (diagrams became cumbersome to draw).

```scheme
(define x (proc-cons 1 2))
```
New environment created - `E1`:
* Enclosing environment = `global`
* Internal bindings:
    * `x` -> `1`
    * `y` -> `2`
* `set-x!` and `set-y!` have pointers out of `E1` to `global`
* `x` in `global` environment is bound to `dispatch` in `E1`

```scheme
(define z (proc-cons x x))
```
New environment created - `E2`:
* Enclosing environment = `global`
* Internal bindings:
    * `x` -> `x` in `global` env
    * `y` -> `x` in `global` env
* `set-x!` and `set-y!` have pointers to `E2`
* `z` in `global` environment is bound to `dispatch` in `E2`  

```scheme
(proc-set-car! (proc-cdr z) 17)
```

`(proc-cdr z)` evaluated:
* New environment created = `E3`:
    * Internal bindings:
        * `z` -> `z` in `global` env
* `(z 'cdr)` evaluated in `E3`:
    * `z` bound to `dispatch` in `E2`
* `(dispatch 'cdr')` evaluated in `E2`:
    * `y` from `E2` is returned = `x` in `global` env
    
`(proc-set-car! x 17)` is evaluated:
* New environment created = `E4`:
    * Internal bindings:
        * `z` -> `x` in `global` env
        * `new-value` -> `17`
* `x` in `global` env bound to `dispatch` in `E1`
* `(dispatch 'set-car!)` evaluated in `E1`:
    * `set-x!` procedure from `E1` returned
    * `(set-x! 17)` evaluated
        * Calls `(set! x 17)` within `E1`
            * Value of `x` in `E1` set to `17`

```scheme
(proc-car x)
```
* `x` in `global` env is bound to `dispatch` in `E1`
* `(dispatch 'car)` evaluated in `E1`
    * `x` from `E1` is returned -> `17`

# Representing Queues
## 3.21

In [39]:
(define (front-ptr queue) (car queue))
(define (rear-ptr queue) (cdr queue))
(define (set-front-ptr! queue item)
  (set-car! queue item))
(define (set-rear-ptr! queue item)
  (set-cdr! queue item))

(define (empty-queue? queue)
  (null? (front-ptr queue)))

(define (make-queue) (cons '() '()))

(define (front-queue queue)
  (if (empty-queue? queue)
      (error "FRONT called with an
             empty queue" queue)
      (car (front-ptr queue))))

(define (insert-queue! queue item)
  (let ((new-pair (cons item '())))
    (cond ((empty-queue? queue)
           (set-front-ptr! queue new-pair)
           (set-rear-ptr! queue new-pair)
           queue)
          (else (set-cdr! (rear-ptr queue)
                          new-pair)
                (set-rear-ptr! queue new-pair)
                queue))))

(define (delete-queue! queue)
  (cond ((empty-queue? queue)
         (error "DELETE! called with
                an empty queue" queue))
        (else (set-front-ptr!
               queue
               (cdr (front-ptr queue)))
              queue)))

In [40]:
(define q1 (make-queue))
(insert-queue! q1 'a)

((a) a)

In [41]:
(insert-queue! q1 'b)

((a b) b)

In [42]:
(delete-queue! q1)

((b) b)

In [5]:
(delete-queue! q1)

(() b)

The Lisp interpreter has no special representation for the compound data structure defined above as a *queue*. Instead, the interpreter will treat them as *lists* of *pairs* - printing them as such. 

Distinct shared data also has no external representation, the value of the variable being pointed to is printed by the interpreter:
```scheme
(define q1 (make-queue))
(insert-queue! q1 'a)
```
```
q1 --> [.|.]----
        |      |
        v <----|
       [.|/]   
        |   
        v
       'a
```
The front and rear pointers of the queue are set to the same variable. The List interpreter will simply print out the value of the variable without distinguishing the fact that they are shared. This makes it seems as if there are two `a`'s in the queue.

`empty-queue?` and `delete-queue` both only interact with the front pointer of the queue. This means that deleting items from a queue until `empty-queue?` returns true leaves the last pair in place being pointed to by the rear pointer of the queue. Therefore, the final structure of `q1` is as follows:
```
q1 --> [.|.] --> [.|.] --> [/|/]
        |         | 
        v         v
       [/|/]     'b
```

In [43]:
(define (print-queue q)
  (display (front-ptr q))
  (newline))

In [44]:
(define q1 (make-queue))
(insert-queue! q1 'a)
(print-queue q1)

(a)


In [45]:
(insert-queue! q1 'b)
(print-queue q1)

(a b)


In [46]:
(delete-queue! q1)
(print-queue q1)

(b)


In [47]:
(delete-queue! q1)
(print-queue q1)

()


## 3.22

In [48]:
(define (make-queue)
  (let ((front-ptr '())
        (rear-ptr '()))
    (define (set-front-ptr! item)
      (set! front-ptr item))
    (define (set-rear-ptr! item)
      (set! rear-ptr item))
    (define (empty-queue?) (null? front-ptr))
    (define (front-queue)
      (if (empty-queue?)
          (error "FRONT called with and empty queue")
          (car front-ptr)))
    (define (insert-queue! item)
      (let ((new-pair (cons item '())))
        (cond ((empty-queue?)
               (set-front-ptr! new-pair)
               (set-rear-ptr! new-pair)
               dispatch)
              (else (set-cdr! rear-ptr new-pair)
                    (set-rear-ptr! new-pair)
                    dispatch))))
    (define (delete-queue!)
      (cond ((empty-queue?)
             (error "DELETE! called with
                    an empty queue" front-ptr))
            (else (set-front-ptr! (cdr front-ptr))
                  dispatch)))
    (define (print) (display front-ptr) (newline))
    (define (dispatch m)
      (cond ((eq? m 'insert-queue!) insert-queue!)
            ((eq? m 'delete-queue!) (delete-queue!))
            ((eq? m 'front-queue) (front-queue))
            ((eq? m 'empty-queue?) (empty-queue?))
            ((eq? m 'print) (print))
            (else (error "Unkown method -- MAKE-QUEUE" m))))
    dispatch))

In [49]:
(define q1 (make-queue))
((q1 'insert-queue!) 'a)
(q1 'print)

(a)


In [50]:
((q1 'insert-queue!) 'b)
(q1 'print)

(a b)


In [51]:
(q1 'delete-queue!)
(q1 'print)

(b)


In [52]:
(q1 'delete-queue!)
(q1 'print)

()


## 3.23

Use *doubly linked list* as this allows $O(1)$ deletion of last element
* Rear pointer of the deque can be set to the *prev* pointer of the last element
    * Avoids $O(1)$ following of pointers from the front of the queue

List element structure:
`((data . prev-ptr) next-ptr)`

In [82]:
(define (make-deque) (cons '() '()))

(define (front-ptr deque) (car deque))
(define (rear-ptr deque) (cdr deque))

(define (set-front-ptr! deque item)
  (set-car! deque item))
(define (set-rear-ptr! deque item)
  (set-cdr! deque item))

(define (empty-deque? deque)
  (null? (front-ptr deque)))

(define (front-queue deque)
  (if (empty-deque? deque)
      (error "FRONT called with an
             empty deque" deque)
      (caar (front-ptr deque))))
(define (rear-queue deque)
  (if (empty-deque? deque)
      (error "REAR called with an
             empty deque" deque)
      (caar (rear-ptr deque))))

(define (front-insert-deque! deque item)
  ;; new element with empty prev and next pointers
  (let ((new-pair (cons (cons item '()) '())))
    (cond ((empty-deque? deque)
           (set-front-ptr! deque new-pair)
           (set-rear-ptr! deque new-pair)
           deque)
          (else 
           ;; set next pointer of new element to current front-ptr
           (set-cdr! new-pair (front-ptr deque))
           ;; set prev pointer of current front-ptr to new element
           (set-cdr! (car (front-ptr deque)) new-pair)
           (set-front-ptr! deque new-pair)
           deque))))

(define (rear-insert-deque! deque item)
  ;; new element with prev pointer of current rear-ptr and empty next pointer
  (let ((new-pair (cons (cons item (rear-ptr deque)) '())))
    (cond ((empty-deque? deque)
           (set-front-ptr! deque new-pair)
           (set-rear-ptr! deque new-pair)
           deque)
          (else
           ;; set next pointer of current rear-ptr to new element
           (set-cdr! (rear-ptr deque) new-pair)
           (set-rear-ptr! deque new-pair)
           deque))))

;; set front-ptr to next pointer of current front-ptr
(define (front-delete-deque! deque)
  (cond ((empty-deque? deque)
         (error "FRONT-DELETE called on
                empty deque" deque))
        (else
         (set-front-ptr! deque (cdr (front-ptr deque)))
         deque)))
;; set rear-ptr to current rear-ptr prev pointer
;; set next pointer of new rear-ptr to empty
(define (rear-delete-deque! deque)
  (cond ((empty-deque? deque)
         (error "REAR-DELETE called on
                empty deque" deque))
        (else
         (set-rear-ptr! deque (cdar (rear-ptr deque)))
         (set-cdr! (rear-ptr deque) '())
         deque)))

(define (print-deque deque)
  (map (lambda (x) (display (car x))(display " ")) (front-ptr deque))(newline))

In [96]:
(define dq (make-deque))
(rear-insert-deque! dq 'a)
(print-deque dq)

a 


In [97]:
(front-insert-deque! dq 'z)
(print-deque dq)

z a 


In [98]:
(rear-insert-deque! dq 'b)
(print-deque dq)

z a b 


In [99]:
(front-insert-deque! dq 'y)
(print-deque dq)

y z a b 


In [100]:
(rear-insert-deque! dq 'c)
(print-deque dq)

y z a b c 


In [101]:
(front-delete-deque! dq)
(print-deque dq)

z a b c 


In [102]:
(rear-delete-deque! dq)
(print-deque dq)

z a b 


In [103]:
(front-delete-deque! dq)
(print-deque dq)

a b 


In [104]:
(rear-delete-deque! dq)
(print-deque dq)

a 


# Representing Tables
## 3.24

In [6]:
(define (make-table same-key?)
  (define (assoc key records)
    (cond ((null? records) false)
          ((same-key? key (caar records))
           (car records))
          (else (assoc key (cdr records)))))
  
  (let ((local-table (list '*table*)))
    (define (lookup key-1 key-2)
      (let ((subtable
             (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record
                   (assoc key-2
                          (cdr subtable))))
              (if record (cdr record) false))
            false)))
    (define (insert! key-1 key-2 value)
      (let ((subtable
             (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record
                   (assoc key-2
                          (cdr subtable))))
              (if record
                  (set-cdr! record value)
                  (set-cdr!
                   subtable
                   (cons (cons key-2 value)
                         (cdr subtable)))))
            (set-cdr!
             local-table
             (cons (list key-1
                         (cons key-2 value))
                   (cdr local-table)))))
      'ok)
    
    (define (dispatch m)
      (cond ((eq? m 'lookup-proc) lookup)
            ((eq? m 'insert-proc!) insert!)
            (else (error "Unknown operation:
                         TABLE" m))))
    dispatch))

(define operation-table (make-table equal?))
(define get (operation-table 'lookup-proc))
(define put (operation-table 'insert-proc!))